In [1]:
import dash
from dash.dependencies import Input, Output
from dash import dcc, html
import plotly.graph_objs as go
from collections import deque
import random
import socket

X = deque(maxlen=30000)
X.append(1)
Y1 = deque(maxlen=3000)
Y1.append(1)
Y2 = deque(maxlen=3000)
Y2.append(2)
Y3 = deque(maxlen=3000)
Y3.append(3)

x_data = [50]
y_data = [50]
z_data = [50]

# Sunucu bilgilerini tanımla
SERVER_IP = '127.0.0.1'
SERVER_PORT = 7545

app = dash.Dash(__name__)
app.layout = html.Div(
    [
        dcc.Graph(id='live-graph', animate=True),
        dcc.Interval(
            id='graph-update',
            interval=1*1000
        ),
    ]
)

# Socket bağlantısı kur
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((SERVER_IP, SERVER_PORT))

@app.callback(Output('live-graph', 'figure'),
              [Input('graph-update', 'n_intervals')])


def update_graph_scatter(n):
    # Her saniyede bir "read*4" komutunu sunucuya gönder
    client_socket.sendall(b"read*4")

    # Sunucudan veri oku
    # Sunucudan veri oku
    response = client_socket.recv(1024)
    response_str = response.decode("utf-8")
    response_int = int(response_str)
    
    # Sunucudan veri oku
    client_socket.sendall(b"read*5")
    response1 = client_socket.recv(1024)
    response_str1 = response1.decode("utf-8")
    response_int1 = int(response_str1)
     # Sunucudan veri oku
    client_socket.sendall(b"read*6")
    response2 = client_socket.recv(1024)
    response_str2 = response2.decode("utf-8")
    response_int2 = int(response_str2)
   
    x_data.append(response_int)
    x_data_1 = x_data.pop()
    y_data.append(response_int1)
    y_data_1 = y_data.pop()
    z_data.append(response_int2)
    z_data_1 = z_data.pop()
    


    # Sunucudan gelen veriyi x_data listesine ekle
    X.append(X[-1] + 1)
    Y1.append(int(x_data_1)   )
    Y2.append(int(y_data_1)  )
    Y3.append(int(z_data_1)   )
    
    
    

    # Scatter plot oluştur
    trace1 = go.Scatter(
        x=list(X),
        y=list(Y1),
        name='temperature_value',
        mode='lines+markers'
    )
    trace2 = go.Scatter(
        x=list(X),
        y=list(Y2),
        name='speed_value',
        mode='lines+markers'
    )
    trace3 = go.Scatter(
        x=list(X),
        y=list(Y3),
        name='voltage_value',
        mode='lines+markers'
    )

    # Grafik düzenini oluştur
    layout = go.Layout(xaxis=dict(range=[min(X), max(X)]),
                       yaxis=dict(range=[min(Y1+Y2+Y3), max(Y1+Y2+Y3)]))

    return {'data': [trace1, trace2, trace3], 'layout': layout}

if __name__ == '__main__':
    app.run(port=7516)
